# Using the Pima Indians Database to predict Diabetes Outcome

The Pima are Native Americans based in Arizona. As a result of changes in diet and physical activity, they have developed a very high incidence of _Type 2 diabetes_. The anonymous medical data used in this notebook was obtained from 768 Pima women. It comprises 8 attributes that might be used to predict diabetes status (the 9th column in the dataset, which is the class to be predicted).

The dataset has been removed from the UCI database, but is still widely available.

We download this data as a csv (comma-separated variable) file (in `../data/pima-indians-diabetes.csv`).

In [ ]:
dataDir = "../data"
dataFile = dataDir + '/pima-indians-diabetes.csv'

# See https://stackoverflow.com/a/82852
import os.path
if not os.path.isfile(dataFile):
  import requests # Remember: you may need to install the requests module: conda install -c anaconda requests 
  
  url = 'https://gist.github.com/chaityacshah/899a95deaf8b1930003ae93944fd17d7/raw/3d35de839da708595a444187e9f13237b51a2cbe/pima-indians-diabetes.csv'
  r = requests.get(url)

  # See https://stackoverflow.com/a/273227
  os.makedirs(dataDir, exist_ok=True)
  with open(dataFile, 'wb') as f:
    f.write(r.content)
    
import pandas as pd
pimaDf = pd.read_csv(dataFile)
pimaDf.head()

While the existing predictor names are descriptive, they are cumbersome when used in models. Therefore, we replace them with simpler predictor names `predNames`.

In [ ]:
print(pimaDf.shape)
predNames = ['NumPreg', 'PlasmaGlucose', 'DiastolicBP', 'SkinFold', 'SerumInsulin', 'BMI', 'PedFn', 'AgeYrs']
className = 'DiabetesClass'
colNames = predNames + [className]

pimaDf.columns = colNames
pimaDf.head()

As can be seen, there are some predictor values that are unrealistic. For example, a `SerumInsulin` measurement of 0 is highly suspicious. similarly, the `SkinFold` test is often used to estimate body fat percentage, so a zero value is similarly unlikely. By contrast, `NumPreg` = 0 is plausible. Therefore, we make the assumption that when one of the other predictors takes a zero value, that the associated measurement is missing, but is assigned zero as a placeholder. In python, it is more meaningful to assign it to `None`, which is how python represents nulls.

In [ ]:
nullablePredNames = predNames.copy()
try:
  nullablePredNames.remove('NumPreg')
except ValueError:
  pass

filteredPimaDf = pimaDf.copy()

for col in nullablePredNames:
  filteredPimaDf.loc[pimaDf[col] == 0, col] = None

filteredPimaDf.head()

The next step, with any predictive model, is to partition the data into training and test subsets. We use the same approach that we used with the Iris Data.

In [ ]:
X = filteredPimaDf[predNames]
y = filteredPimaDf[['DiabetesClass']]

# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training and test sets
def splitData(X, y):
  testSize=0.33
  randomState=5
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize, stratify=y, random_state=randomState)
  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = splitData(X, y)
print('Split {0} rows into train={1} and test={2} rows'.format(len(pimaDf.index), len(X_train.index), len(X_test.index)))

## Naive Bayes from first principles

The following python code was extracted from [How To Implement Naive Bayes From Scratch in Python](https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/).

In [ ]:
# Example of Naive Bayes implemented from Scratch in Python
# Based on https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/
import math
import numpy as np

def separateByClass(dataset):
  # Given a dataset, return a dict named _separated_
  # The dict keys are the class labels
  # The dict value for each key lis a list of the rows associated with that key
  separated = {}
  for index, row in dataset.iterrows():
    # Note that row[-1] is the last element in the row and is the assigned class label
    label = row[-1]
    if (label not in separated):
      # This is the first time we have met this class label, so create an empty list
      separated[label] = []
    separated[label].append(row)
  return separated

def summarize(dataset):
  # Create (mean, stdev) tuples for each column in the given dataset, returning it as a list
  summaries = [(np.mean(attribute), np.std(attribute)) for attribute in zip(*dataset)]
  # remove the last element of the list, because we are not interested in the summarised class labels
  del summaries[-1]
  return summaries

def summarizeByClass(dataset):
  # Use separateByClass to generate the subsets of the rows by classLabel
  # Then compute the list of (mean,stdev) tuples for each predictor, for each classLabel
  separated = separateByClass(dataset)
  summaries = {}
  for classValue, instances in separated.items():
    summaries[classValue] = summarize(instances)
  return summaries

def calculateProbability(x, mean, stdev):
  # Compute the Gaussian(mean,stdev) probability evaluated at x
  exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
  return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, inputVector):
  # Given the training summaries, and a row from the testSet,
  # compute the probability of each of the class labels
  probabilities = {}
  # Loop over the classValues...
  for classValue, classSummaries in summaries.items():
    probabilities[classValue] = 1
    # Loop over the summarised predictors
    for i in range(len(classSummaries)):
      mean, stdev = classSummaries[i]
      # Note that x is the value of the associated predictor in the test instance
      x = inputVector[i]
      # Now calculate the Gaussian probability for this x
      # We accumulate this predictor's probability by multiplying it with all the others
      probabilities[classValue] *= calculateProbability(x, mean, stdev)
  return probabilities
      
def predict(summaries, inputVector):
  # Given the training summaries, and a row from the testSet,
  # compute the probability of each of the class labels
  probabilities = calculateClassProbabilities(summaries, inputVector)
  bestLabel, bestProb = None, -1
  # Now look for the bestLabel, which is associated with the highest probability
  for classValue, probability in probabilities.items():
    if bestLabel is None or probability > bestProb:
      bestProb = probability
      bestLabel = classValue
  return bestLabel

def getPredictions(summaries, testSet):
  # Given (mean,stdev) summaries (by predictor, by classLabel) from the training data,
  # compute predicted classLabels for each row in testSet, storing them in a list
  predictions = []
  for index, row in testSet.iterrows():
    result = predict(summaries, row)
    # Add the predicted classLabel to the predictions list
    # Strictly, predictions should be a pandas series, so its index can match that of the testSet
    predictions.append(result)
  return predictions

def getAccuracy(testSet, predictions):
  # Compute the accuracy as a % of correct predictions over all predictions
  correct = 0
  i = 0
  for index, row in testSet.iterrows():
    # Accumate the cases where the assigned classLabel matches the predicted classLabel
    if row[-1] == predictions[i]:
      correct += 1
    i += 1
  return (correct/float(len(testSet))) * 100.0



In [ ]:
# prepare model
trainingSet = X_train.assign(DiabetesClass=y_train)
summaries = summarizeByClass(trainingSet)
# test model
testSet = X_test.assign(DiabetesClass=y_test)
predictions = getPredictions(summaries, testSet)
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: {0}%'.format(accuracy))

As can be seen, the accuracy is approximately 65%.  We might be able to improve this by removing predictors having low prediction significance, computing functions of the predictors, etc., but such techniques are more easily done when using the scikit-learn implementation.

## Using scikit-learn to compare Naive Bayes and SVM classifiers

Needless to say, it is not necessary to program classifiers like Naive Bayes and SVM manually, because scikit-learn versions exist.

Given the fact that there are so many predictors (8, though often the number of predictors (e.g., keywords for spam detection, or medium-resolution images for "cat" recognition....) can number tens of thousands), it is often advisable to look for a) correlations between predictors and b) correlations between predictors and the assigned class labels.

In [ ]:
corr = pimaDf.corr()
corr

Note that some predictors appear to be moderately correlated with each other, notably corr(`PlasmaGlucose`,`serumInsulin`) = 0.33, which is quite high, but might be expected. We note that corr(`PlasmaGlucose`,`DiabetesClass`) = 0.47, which is the highest of all such correlations, therefore `plasmaGlucose` is quite highly correlated with diabetes state, as might be expected.

We can also plot histograms of the predictor values, where we see that 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

pimaDf.hist(bins=50, figsize=(20, 15))
plt.show()

We can see the general distribution for each column and can see the effects of the missing values too. Apart from assigning such missing values to None, there is another way of accommodating such missing data, which is to impute values are statistically "neutral". We do this by assigning, when the predictor value is zero, the median of the remaining values of a predictor.

In [ ]:
adjustedPimaDf = pimaDf.copy()

for col in nullablePredNames:
  # Calculate the median value
  medianVal = pimaDf.loc[pimaDf[col] != 0][col].median()
  adjustedPimaDf.loc[pimaDf[col] == 0, col] = medianVal

adjustedPimaDf.head()

As before, we decide to split the dataset into training and test data.

In [ ]:
X = adjustedPimaDf[predNames]
y = adjustedPimaDf[['DiabetesClass']]

X_train, X_test, y_train, y_test = splitData(X, y)
print('Split {0} rows into train={1} and test={2} rows'.format(len(pimaDf.index), len(X_train.index), len(X_test.index)))

One of the things that is noticeable is the fact that different predictors have different ranges of values, e.g., `NumPreg` is typically less than 10 while `SerumInsulin` is often greate than 100. `NaiveBayes` is based on probability so the different prediction values do not matter, other classifiers like SVM are sensitive to feature scaling. Luckily, scikit-learn has the ability to scale predictors so that they use a common scale.

In [ ]:
# Apply a scaler
from sklearn.preprocessing import MinMaxScaler as Scaler

scaler = Scaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Now we can compare `NaiveBayes` and `SVM` classifiers. First we prepare for the comparison, which will use k-fold cross-validation rather than simply quoting a single accuracy score.

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Import the slearn utility to compare algorithms
from sklearn import model_selection

# Prepare an array with the 2 algorithms
models = []
models.append(('NB', GaussianNB()))
models.append(('SVC', SVC(gamma='auto')))

# Prepare the configuration to run the test
seed = 42
results = []
names = []
nSplits = 10

Now we run the comparison...

In [ ]:
import numpy as np

# Fit the classifiers and output the crossVal results
for name, model in models:
    # We use k-fold cross validation, where 10% of the training data is held back each time 
    kfold = model_selection.KFold(n_splits=nSplits, random_state=seed, shuffle=True)
    crossValResults = model_selection.cross_val_score(
        model, X_train_scaled, np.ravel(y_train), cv=kfold, scoring='accuracy')
    results.append(crossValResults)
    names.append(name)
    # Now print the results
    print("%s: %f (%f)" % (name, crossValResults.mean(), crossValResults.std()))

One of the advantages of k(=10)-fold cross-validation is that we get not just one estimate of accuracy, but 10 (in this case). Therefore we can look at how the accuracy is distributed.

In [ ]:
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

Although it is not very obvious here, SVM generally outperforms Naive Bayes.

In [ ]:
# Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

def showDiagnostics(y, y_pred):
  # Model Accuracy, how often is the classifier correct?
  print("Accuracy:",accuracy_score(y, y_pred))
  print("Confusion Matrix:")
  print(confusion_matrix(y, y_pred))
  print("Classification Report:")
  print(classification_report(y, y_pred, digits=3))



In [ ]:
# Create a Naive Bayes Classifier
clf=GaussianNB()

# Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train_scaled,np.ravel(y_train))

# prediction on test set
y_pred=clf.predict(X_test_scaled)

showDiagnostics(y_test, y_pred)

In [ ]:
# Create a SVM Classifier
clf=SVC(gamma='auto')

# Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train_scaled,np.ravel(y_train))

# prediction on test set
y_pred=clf.predict(X_test_scaled)

showDiagnostics(y_test, y_pred)

The traditional "compare assigned and predicted class labels for the test set" gives results that are consistent with the k-fold cross-validation results earlier.